# 联邦平均

联邦学习（Federated Learning）是一种安全分布式深度学习技术，它允许各个数据持有者在不公开数据的情况下协同训练得到一个共享的模型，其目的是打破数据孤岛，在保护数据的隐私的前提下利用数据实现数据整合。
目前关于联邦学习的实现有许多说法，有梯度聚合、模型平均、选择上传等等。有的认为参数服务器持有模型，参与者不持有；有的认为是各个数据持有者持有模型，参数服务器不需要获取模型。众说纷纭。
但其核心是不变的：那就是数据分离，通信加密。

联邦学习的各个参与者，会在本地训练模型，然后每一轮（或者固定间隔的轮次）将其模型参数，或者梯度（广义梯度，即前一轮次与当前轮次的模型参数的差）上传到参数服务器，由参数服务器将各个参与者的上传参数进行聚合，得到的结果再返还给各个参与者，参与者更新本地模型后，继续训练。
在这个过程中，有如下几个计划：

模型训练，模型是需要训练的，这个操作必须由各个参与者执行
安全聚合，在梯度传递到参数服务器并返回给各个参与者这个过程中，传递的参数是不安全的，需要进行加密保护；并且，参数服务器要对参数进行聚合。


In [2]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

## 步骤1 创建数据的所有者

In [3]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

## 步骤2 创建模型

In [4]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

## 步骤3 将模型发送给Alice和Bob

> 1. 创建模型，前项传播计算中间结果，计算损失值，损失值反向传播形成梯度，使用优化器进行梯度下降。
> 2. 在联邦学习模型中，聚合的是模型的权重，而不是模型的梯度。是经过一次梯度下降操作后的模型的权重。
> 3. adam优化器中的参数包括两个部分：一个是动量梯度计算，一个是RMscrop。都是由历史的梯度得出来的。
> 4. 所以要想服务器知道模型的adam优化器的结果，必须将历史的梯度也进行上传，显然不合理。

In [5]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

## 步骤4 并行训练Bob和Alice的模型

In [6]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.1430) Alice:tensor(1.3841)
Bob:tensor(0.0712) Alice:tensor(0.0938)
Bob:tensor(0.0492) Alice:tensor(0.0699)
Bob:tensor(0.0394) Alice:tensor(0.0582)
Bob:tensor(0.0330) Alice:tensor(0.0484)
Bob:tensor(0.0280) Alice:tensor(0.0403)
Bob:tensor(0.0239) Alice:tensor(0.0335)
Bob:tensor(0.0204) Alice:tensor(0.0279)
Bob:tensor(0.0174) Alice:tensor(0.0232)
Bob:tensor(0.0148) Alice:tensor(0.0193)


## 步骤5 客户端发送模型到服务器

In [7]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

## 步骤6 模型平均

In [8]:
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())


In [10]:
print(model._parameters)

OrderedDict([('weight', Parameter containing:
tensor([[0.6258, 0.1664]], requires_grad=True)), ('bias', Parameter containing:
tensor([-0.0167], requires_grad=True))])


## 步骤7 迭代以上步骤

In [11]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0046) Alice:tensor(0.0139)
Bob:tensor(0.0013) Alice:tensor(0.0066)
Bob:tensor(0.0003) Alice:tensor(0.0030)
Bob:tensor(4.8089e-05) Alice:tensor(0.0014)
Bob:tensor(4.9928e-05) Alice:tensor(0.0006)
Bob:tensor(9.4057e-05) Alice:tensor(0.0003)
Bob:tensor(0.0001) Alice:tensor(0.0001)
Bob:tensor(0.0001) Alice:tensor(7.4250e-05)
Bob:tensor(0.0001) Alice:tensor(3.8461e-05)
Bob:tensor(0.0001) Alice:tensor(2.0683e-05)


In [12]:
# 验证模型
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [13]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0356],
        [0.0321],
        [0.9589],
        [0.9553]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0060)
